# How to Build Secure Socket Layer Checker in Python ?

Install the Follow Dependencies
!pip install OpenSSL
!pip install datetime
!pip install cryptography
!pip install idna 

idna stands for : ( Internationalized Domain Names in Applications (IDNA))

In [ ]:
from OpenSSL import SSL
import datetime
import time
from cryptography import x509
from cryptography.x509.oid import NameOID
import idna

from socket import socket
from collections import namedtuple

# You need to Provide your URL for which you are checking SSL

In [ ]:
HostInfo = namedtuple(field_names='cert hostname peername', typename='HostInfo')

HOSTS = [
     ('mysite.com', 443),
     ('yoursitename.org', 443),
    ('xyz.co.in', 443),
    # ('', 443),
    #('faß.de', 443),
    #('favourite.com', 443),

## Here it  verify  CA(certificate authority) trusted, servername/hostname/Server Name Indication
##  service_identity.pyopenssl.verify_hostname(client_ssl, hostname) 
       

In [ ]:
def verify_cert(cert, hostname):

    cert.has_expired()
    

In [ ]:
def get_certificate(hostname, port):
    
    # first we need to ENCODE the HostName eg( mywebsite.com)that we have Provided.
    
    hostname_idna = idna.encode(hostname)
    
    # Then we need to Establish Connection with the server So we Use SOCKET
    
    sock = socket()
    
    #To connect with server in-build function named as Connect() in SOCKET 
    #here we Pass two parameter:
    # Hostname - is the URl that you Provided above 
    # Port  - to show it on your Laptop/Desktop, We need to Define a port where it will get the information back to us.
    
    sock.connect((hostname, port))
    #getpeername() function shall retrieve the peer address of the specified socket
    # Return peername as weed it as Output.
    peername = sock.getpeername()
    ctx = SSL.Context(SSL.SSLv23_METHOD) # most compatible
    ctx.check_hostname = False
    ctx.verify_mode = SSL.VERIFY_NONE

    sock_ssl = SSL.Connection(ctx, sock)
    sock_ssl.set_connect_state()
    
    #here set_tlsext_host_name
    # tlsext - is transport layer security extension THAT SENDS YOUR hostname_idna i.e
    #(i.e your website URL which you have Encoded above.)
    
    sock_ssl.set_tlsext_host_name(hostname_idna)
    sock_ssl.do_handshake()
    cert = sock_ssl.get_peer_certificate()
    # Return crypto_cert as we need it as output.
    crypto_cert = cert.to_cryptography()
    sock_ssl.close()
    sock.close()

    return HostInfo(cert=crypto_cert, peername=peername, hostname=hostname)

# How identification is done on Web for any website or web-app ?

##### SSL/TLS X.509 certificates are digital files that are used for Secure Sockets Layer (SSL) or Transport Layer Security (TLS). An SSL/TLS certificate is one of the most popular types of X.509 certificates, or a type of public key certificate which uses the X.509 standard. X.509 certificates contain a public key and the identity of a hostname, organization, or individual. 

## To Identify the hostname, organization, or individual - It requires X.509 certificates contain a public key and identifies the Same.

##### To Get the DNS NAME we Need to Provide the  ( Crypto_cert  ) as its Input , We have made it Return in the Above Funtion as 'Cert'

In [ ]:
def get_alt_names(cert):
    try:
        ext = cert.extensions.get_extension_for_class(x509.SubjectAlternativeName)
        # here it will return the IP Address of Cert we provided 
        return ext.value.get_values_for_type(x509.DNSName)
    except x509.ExtensionNotFound:
        return None

def get_common_name(cert):
    try:
        names = cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)
        return names[0].value
    except x509.ExtensionNotFound:
        return None

    # Here it returns the Issuer of the Certificate Lets Say  "Let's Encrypt"  https://letsencrypt.org/
def get_issuer(cert):
    try:
        names = cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)
        return names[0].value
    except x509.ExtensionNotFound:
        return None

Now All we need to do is to Print all the Stuff !!

In [ ]:
def print_basic_info(hostinfo):
    s = '''» {hostname} « … {peername}
    \tcommonName: {commonname}
    \tSAN: {SAN}
    \tissuer: {issuer}
    \tnotBefore: {notbefore}
    \tnotAfter:  {notafter}
    '''.format(
            hostname=hostinfo.hostname,
            peername=hostinfo.peername,
            commonname=get_common_name(hostinfo.cert),
            SAN=get_alt_names(hostinfo.cert),
            issuer=get_issuer(hostinfo.cert),
            notbefore=hostinfo.cert.not_valid_before,
            notafter=hostinfo.cert.not_valid_after
    )
    timenow = datetime.datetime.now()
    if timenow <= hostinfo.cert.not_valid_after :
        print("certfication is secured")
    else:
        print("SSL certfication has expired")

    print(s)

def check_it_out(hostname, port):
    hostinfo = get_certificate(hostname, port)
    print_basic_info(hostinfo)

In [ ]:

def check_it_out(hostname, port):
    hostinfo = get_certificate(hostname, port)
    print_basic_info(hostinfo)


import concurrent.futures
if __name__ == '__main__':
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as e:
        for hostinfo in e.map(lambda x: get_certificate(x[0], x[1]), HOSTS):
            print_basic_info(hostinfo)

# Compelete Code

In [ ]:

from OpenSSL import SSL
import datetime
import time
from cryptography import x509
from cryptography.x509.oid import NameOID
import idna

from socket import socket
from collections import namedtuple

HostInfo = namedtuple(field_names='cert hostname peername', typename='HostInfo')

HOSTS = [
# Enter your Website URL '' Below.
     ('', 443),
     ('', 443),
    ('mywebsite.com', 443),
    # ('', 443),
    #('faß.de', 443),
    #('самодеј.мкд', 443),
]

def verify_cert(cert, hostname):
    # verify notAfter/notBefore, CA trusted, servername/sni/hostname
    cert.has_expired()
    # service_identity.pyopenssl.verify_hostname(client_ssl, hostname)
    # issuer

def get_certificate(hostname, port):
    hostname_idna = idna.encode(hostname)
    sock = socket()

    sock.connect((hostname, port))
    peername = sock.getpeername()
    ctx = SSL.Context(SSL.SSLv23_METHOD) # most compatible
    ctx.check_hostname = False
    ctx.verify_mode = SSL.VERIFY_NONE

    sock_ssl = SSL.Connection(ctx, sock)
    sock_ssl.set_connect_state()
    sock_ssl.set_tlsext_host_name(hostname_idna)
    sock_ssl.do_handshake()
    cert = sock_ssl.get_peer_certificate()
    crypto_cert = cert.to_cryptography()
    sock_ssl.close()
    sock.close()

    return HostInfo(cert=crypto_cert, peername=peername, hostname=hostname)

def get_alt_names(cert):
    try:
        ext = cert.extensions.get_extension_for_class(x509.SubjectAlternativeName)
        return ext.value.get_values_for_type(x509.DNSName)
    except x509.ExtensionNotFound:
        return None

def get_common_name(cert):
    try:
        names = cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)
        return names[0].value
    except x509.ExtensionNotFound:
        return None

def get_issuer(cert):
    try:
        names = cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)
        return names[0].value
    except x509.ExtensionNotFound:
        return None


def print_basic_info(hostinfo):
    s = '''» {hostname} « … {peername}
    \tcommonName: {commonname}
    \tSAN: {SAN}
    \tissuer: {issuer}
    \tnotBefore: {notbefore}
    \tnotAfter:  {notafter}
    '''.format(
            hostname=hostinfo.hostname,
            peername=hostinfo.peername,
            commonname=get_common_name(hostinfo.cert),
            SAN=get_alt_names(hostinfo.cert),
            issuer=get_issuer(hostinfo.cert),
            notbefore=hostinfo.cert.not_valid_before,
            notafter=hostinfo.cert.not_valid_after
    )
    timenow = datetime.datetime.now()
    if timenow <= hostinfo.cert.not_valid_after :
        print("certfication is secured")
    else:
        print("SSL certfication has expired")

    print(s)

def check_it_out(hostname, port):
    hostinfo = get_certificate(hostname, port)
    print_basic_info(hostinfo)


import concurrent.futures
if __name__ == '__main__':
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as e:
        for hostinfo in e.map(lambda x: get_certificate(x[0], x[1]), HOSTS):
            print_basic_info(hostinfo)
